# Reading csv file into panda dataframes and converting dataframe into json file

In [1]:
import pandas as pd
import json
df1 = pd.read_csv("E:/CS587/Food_Inspections.csv")

In [2]:
s = df1.iloc[:100000].to_json(orient='records')

In [3]:
cont = json.loads(s)

In [4]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()

# Inserting the data into Elastic Search

In [5]:
i = 0
actions = []
for data in cont:
    action = {
        "_index": "food_inspection",
        "_type": "food",
        "_id": i,
        "_source": {
            "any":data
            }
        }
    actions.append(action)
    i += 1

helpers.bulk(es, actions)

(100000, [])

# Qurey to fetch the data from elastic Search

In [6]:
doc = {
        'size' : 10000,
        'query': {
            'match_all' : {}
       }
   }
res = es.search(index='food_inspection', body=doc,scroll='1h')

In [7]:
sid = res['_scroll_id']
scroll_size = res['hits']['total']

# Populating the data into local variable

In [8]:
final_location_ll = []
while(scroll_size > 0):

    for doc in res['hits']['hits']:
        location_ll = []
        results = doc['_source']['any']
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if 'Address' in results.keys():
                    if(results['Latitude'] != None and results['Longitude'] != None  and results['Address'] != None):

                        
                        location_ll.append(float(results['Latitude']))
                        location_ll.append(float(results['Longitude']))
                        final_location_ll.append(location_ll)
    res = es.scroll(scroll_id = sid, scroll = '2m')
    sid = res['_scroll_id']
    scroll_size = len(res['hits']['hits'])  

In [10]:
len(final_location_ll)

99656

# Importing folium to plot the map

In [11]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

print(folium.__version__)

0.5.0


In [12]:
m = folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)
m

# For generating HeatMap

In [13]:
m.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
m